In [102]:
# Pre-Process CMV Data
# ====================

import pandas as pd
import json
import re
from IPython.display import Markdown
import random

In [33]:
cmv = [json.loads(ln) for ln in open("./data/pairs.jsonl", "r")]

In [114]:
id_ = []
claims = []
args = []
counters = []

for _ in cmv:
    id_.append(_["submission"]["id"])
    claims.append(_["submission"]["title"])
    args.append(_["submission"]["selftext"])
    counters.append(_["delta_comment"]["comments"][0]["body"])

In [116]:
_ = random.randint(0, len(cmv))

print(_, "/", len(counters))
counters[_]

9890 / 10303


"&gt; It could be a 1000 years, or maybe in 5000, but eventually you will have mastered everything and no longer have anything left to do with your life.\n\nIf you get to the point that you feel there is literally nothing else to do with your life what is stopping you from ending it through means such as assisted suicide? I'm sure I'm not the only one but mastering everything would not be that important to me. The part of immortality that appeals most to me is being able to stick around just to see the amazing things humanity might one day achieve. \n\n&gt; If immortality was achieved we would have to stop having children. There we be no more new lives born into this world. Would you really want to live in a world where there can be no new life?\n\nWho says we have to stop having children? Sure being biologically immortal would vastly reduce the death rate, however there are still many ways that people could die such as natural disasters/murder/suicide/stupidity.\n\nAnd who's to say th

In [118]:
print(len(id_))

10303


In [119]:
# Clean Post
def cleanup(cmv_post):
    lines = [
        line for line in cmv_post.splitlines()
        if not line.lstrip().startswith("&gt;")
        and not line.lstrip().startswith("____")
        and "edit" not in " ".join(line.lower().split()[:2])
    ]
    return "\n".join(lines)

# Display Post IPython Markdown
def show_post(title, cmv_post, counter):
    cmv_post = cleanup(cmv_post)
    md_format = "**{}** \n \n {} \n \n **Counter** \n \n {}".format(title, cmv_post, counter)
    md_format = "\n".join(["> " + line for line in md_format.splitlines()])

    return Markdown(md_format)

# Clean Text
def full_clean(data):
    # TODOs: Consider .strip()
    cleaned = []

    for i in data:
        # CMV clean-up
        clean = cleanup(i)

        # RegEx clean-up
        clean = re.sub("CMV:", " ", clean)
        clean = re.sub("CMV", " ", clean)
        clean = re.sub(r"http\S+", "", clean)
        clean = re.sub(r"\n", "", clean)
        clean = re.sub(r'(?<=[a-z])\'(?=[a-z])', '', clean)
        clean = re.sub('([^a-zA-Z\s.!?])', "", clean)
        clean = re.sub('\s+', ' ', clean)

        clean = re.sub(r"www\S+", "", clean)
        cleaned.append(clean.strip())

        #clean = re.sub("^\s", "", clean)

    return cleaned

In [122]:
### CONSTRUCT DATAFRAME OBJECTS ###

args_obj = {
    "id": id_,
    "claim": claims,
    "argument": args,
    "counter": counters
}

args_df = pd.DataFrame(args_obj).astype(str)
args_df

,id,claim,argument,counter
0,t3_30oi71,CMV: We should strengthen the traditional safe...,## Section I: Why is Basic Income Increasingly...,The majority of your points seem predicated on...
1,t3_30oi71,CMV: We should strengthen the traditional safe...,## Section I: Why is Basic Income Increasingly...,&gt; Section V: Does the Welfare Trap Truly Ex...
2,t3_30oi71,CMV: We should strengthen the traditional safe...,## Section I: Why is Basic Income Increasingly...,"First, your disagreements about different fund..."
3,t3_30oi71,CMV: We should strengthen the traditional safe...,## Section I: Why is Basic Income Increasingly...,your position rests on the assumption that mos...
4,t3_4gdj35,CMV: Males and Females are socially and emotio...,Hi CMV! \n\nI am of the belief that there cann...,You seem to be basing your entire argument on ...
...,...,...,...,...
10298,t3_3h69kp,"CMV:The criticism of ""nice guys"" (as seen in r...",The reasoning behind this view is that everyon...,&gt; everyone is entitled to human companionsh...
10299,t3_3h69kp,"CMV:The criticism of ""nice guys"" (as seen in r...",The reasoning behind this view is that everyon...,"Hey OP, so what WOULD change your view?\nAre y..."
10300,t3_1tqlde,"I believe that people who say ""Why don't you j...",There have been a number of discussions on var...,If you remove the entire religious aspect of C...
10301,t3_1qiccr,I don't see the point of feminism in the West....,Feminism seems to me to be about acquiring equ...,&gt;Feminism seems to me to be about acquiring...


In [3]:
# ### CONSTRUCT DATAFRAME OBJECTS ###
#
# arg_load = []
# for line in open('../data/train_cmv.jsonlist', 'r'):
#     arg_load.append(json.loads(line))
#
# args = pd.DataFrame(arg_load)
# titles = args["op_title"]
# props = args["op_text"]
# id = args["op_name"]
#
# wins = [
#     args["positive"][i]["comments"][0]["body"] for i in range(0, len(args))
# ]
#
# debate = {
#     "id": id,
#     "Titles": titles,
#     "Arguments": props,
#     "Counters": wins
# }
#
# debate = pd.DataFrame(data = debate, columns = ["id", "Titles", "Arguments", "Counters"]).astype(str)

In [123]:
# Exploritory Keyword Search
keyword = "Philosophy"
args_df[args_df['counter'].str.contains(keyword,case=False)]

,id,claim,argument,counter
9,t3_5aceoz,CMV: Apple are falsely equating simplicity wit...,"**Update**\n\nThanks for all the replies, ther...",I'm not so much trying to change your view wit...
349,t3_31kfkt,"CMV: It is MY right to decide my fate, and tha...",***EDIT (please read)***: thank you all for yo...,I won't try to change your view that ones righ...
363,t3_2c0frm,CMV: /r/atheism should be renamed to /r/antith...,If you go to /r/atheism or even /r/atheismrebo...,There already is an /r/Antitheism which gets m...
499,t3_6y8y6j,CMV: All our motivations ultimately do (and ma...,"In my opinion, all our motivations and instinc...",Well slight correction here that actually is h...
507,t3_4zoqt4,CMV: Suicide by Aging is a thing.,"So, I have this very acid opinion of the Abrah...",OK. Freed from the blinders of my suicide cul...
...,...,...,...,...
10025,t3_4gmeoo,CMV: Black people need to begin accepting thei...,**[This post is politically incorrect. This po...,While a large portion of people have made very...
10041,t3_5qv6d6,CMV: There is no useful role for God/religion ...,The concept of a creator explained why things ...,I'm going to need several sources and bibliogr...
10044,t3_5ohwa6,CMV: I don't understand people who deny evolution,What about the bacterias? Why do we get sick? ...,I was raised a creationist. I denied evolution...
10104,t3_1kmwmn,I believe Conservative stances and policies ar...,"My overall problem with conservatism, whether ...","You're half right. Generally speaking, conserv..."


In [124]:
# Explore Post
show_post(args_df["claim"][5], args_df["argument"][5], args_df["counter"][5])

> **CMV: Males and Females are socially and emotionally incompatible and heterosexual relationships are unrealistic and farcical.** 
>  
>  Hi CMV! 
> 
> I am of the belief that there cannot exist a legitimate emotional bond between a male and a female that lead to a loving relationship (that is - one of romance). Not once have I seen a couple that did not have an enormous gap in interests, composure, speech patterns, and emotional engagement. There's always and awkward disparity between each partner, and they seem to be tolerating each other, whilst there is little to grasp on why they are interested in each other aside from basic biology. 
> 
> As a result of this, homosexual relationships have always seemed purely the logical and socially optimal choice for anyone pursuing a relationship, and heterosexual relationships continue only because of a heteronormitive society and the biology of reproduction (which I believe is immoral, but that's another topic). I may be biased, but from my perspective and experience I've simply never seen a straight couple that was legitimately compatible in most ways. 
>  
>  **Counter** 
>  
>  &gt; Not once have I seen a couple that did not have an enormous gap in interests
> 
> For what it's worth, there are two large biases here with you need to contend with.
> 
> 1. The couples which you know and see will be influenced heavily with who you associate with and is not representative of relationships in general or the potential which some relationships can reach
> 
> 2. This also depends heavily upon your judgment of those relationships. Which has the following problems 
> 
>   * You might not have complete indepth knowledge between the dynamics of these couples
> 
>   * Even if you did, you might midjudge aspects of a relationship. Do you know exactly what the other person needs or is looking for? Do you know what elements create successful relationships? Can you judge those correctly 100% of the time?
> 
> Also,
> 
> &gt; and heterosexual relationships continue only because of a heteronormitive society and the biology of reproduction 
> 
> What about relationships which last past this stage? This statement obviously doesn't hold up either.
> 
> &gt; I may be biased, but from my perspective and experience I've simply never seen a straight couple that was legitimately compatible in most ways.
> 
> That's basically your argument "i've never seen it work before therefore it cant work". It's not a very good position to defend because, as you've already awknowledged, its covered in bias which prevents it from being logically valid.
> 
> 
> 
> ___
> 
> I'm personally in a working relationship and the differences between thinking which come from being different sexes is quite apparent, that doesn't mean we cant work with it though.
> 
> &gt;they seem to be tolerating each other
> 
> Yes, there's a lot of that in a relationship. After all of the love hormones wear off after a couple of years there is an element of toleration since your partners flaws begin to show and you become to get to know them. The point is though that we're also able to accept each others flaws and let on with our lives despite them.
> 
> The reason why we can ignore them is because more of the time we are supportive of each other. We communicate openly and honestly and we are good listeners. We enjoy each others company much more often than not. We also accept that if the other falls short for some reason that they are only human and that's to be expected.

In [127]:
titles_clean = full_clean(claims)
args_clean = full_clean(args)
counters_clean = full_clean(counters)

args_clean = {
    "id": id_,
    "claim": titles_clean,
    "argument": args_clean,
    "counter": counters_clean
}

args_clean_df = pd.DataFrame(args_clean)
args_clean_df

,id,claim,argument,counter
0,t3_30oi71,We should strengthen the traditional safety ne...,Section I Why is Basic Income Increasingly Pop...,The majority of your points seem predicated on...
1,t3_30oi71,We should strengthen the traditional safety ne...,Section I Why is Basic Income Increasingly Pop...,Try searching for welfare trap welfare cliff u...
2,t3_30oi71,We should strengthen the traditional safety ne...,Section I Why is Basic Income Increasingly Pop...,First your disagreements about different fundi...
3,t3_30oi71,We should strengthen the traditional safety ne...,Section I Why is Basic Income Increasingly Pop...,your position rests on the assumption that mos...
4,t3_4gdj35,Males and Females are socially and emotionally...,Hi ! I am of the belief that there cannot exis...,You seem to be basing your entire argument on ...
...,...,...,...,...
10298,t3_3h69kp,The criticism of nice guys as seen in rniceguy...,The reasoning behind this view is that everyon...,Why? Everyone should have it and society shoul...
10299,t3_3h69kp,The criticism of nice guys as seen in rniceguy...,The reasoning behind this view is that everyon...,Hey OP so what WOULD change your view?Are you ...
10300,t3_1tqlde,I believe that people who say Why dont you jus...,There have been a number of discussions on var...,If you remove the entire religious aspect of C...
10301,t3_1qiccr,I dont see the point of feminism in the West.,Feminism seems to me to be about acquiring equ...,This is not complete its not just about legal ...


In [131]:
# Explore Post
show_post(args_clean_df["id"], args_clean_df["argument"][_], args_clean_df["counter"][_])

> **0        t3_30oi71
> 1        t3_30oi71
> 2        t3_30oi71
> 3        t3_30oi71
> 4        t3_4gdj35
>            ...    
> 10298    t3_3h69kp
> 10299    t3_3h69kp
> 10300    t3_1tqlde
> 10301    t3_1qiccr
> 10302    t3_1bc54q
> Name: id, Length: 10303, dtype: object** 
>  
>  Ive been reading up on a biologist named Aubrey de Grey and how he thinks humans can achieve biological immortality and we should do it. I am not sure this is a good idea. I am all for life extension if possible. Living to or even a years like the Asari from the video game Mass Effect would be a good thing.But immortality would be more hell than a good thing. If one is immortal then you have all the time in the world to do things. But eventually I think you will run out of things to do. It could be a years or maybe in but eventually you will have mastered everything and no longer have anything left to do with your life.What if your loved ones somehow die because someone killed them and thus immortality didnt help them? Would you really want to live forever after that?Some would argue that death pushes us forward to do things because we know we have a time limit. Im not so sure of this argument myself. I enjoy life and go to do things not because I have a time limit but because I enjoy doing these things. Nevertheless it is an argument that some has brought up.If immortality was achieved we would have to stop having children. There we be no more new lives born into this world. Would you really want to live in a world where there can be no new life?And finally if immortality was possible do you really think you or I would get it? No the rich and powerful will have access to it. Hitlers year Reich may very well be a possibility. If the rich and dictators achieve immortality then we would be crush under their boots. Anyone ever seen the movie In Time? It would be a lot like that.I think we should extend our lifespans if it is even possible but immortality should be out of the question. Nobody should live forever. 
>  
>  **Counter** 
>  
>  If you get to the point that you feel there is literally nothing else to do with your life what is stopping you from ending it through means such as assisted suicide? Im sure Im not the only one but mastering everything would not be that important to me. The part of immortality that appeals most to me is being able to stick around just to see the amazing things humanity might one day achieve. Who says we have to stop having children? Sure being biologically immortal would vastly reduce the death rate however there are still many ways that people could die such as natural disastersmurdersuicidestupidity.And whos to say that we have to be limited to living on this world? There are already people working towards goals such as colonizing mars. Now just imagine if you give these people thousands of years to focus on their work. I dont think its too far fetched to imagine that we could eventually colonize multiple different planets thus creating a need for more people to be born.

In [133]:
# Exploritory Search Keywords; Assert Clean (URLs: 'http', 'www')
keyword = "www"
args_clean_df[args_clean_df['counter'].str.contains(keyword,case=False)]

,id,claim,argument,counter


In [134]:
# Search Keywords; Assert missing values at start of sentence (known argument issue)
keyword = "harassment"
args_clean_df[args_clean_df['counter'].str.contains(keyword,case=False)]

,id,claim,argument,counter
354,t3_1i2u57,I believe that kids who are bullied are partia...,So since youre reading this you probably haven...,I was unaware that having puberty later than o...
421,t3_3ia43m,It is neither derogatory nor a promotion of se...,I believe that in our current society heterose...,Harassment is nothing more than aggressive pre...
582,t3_1zwqb4,I think that crimes committed while acting in ...,I personally think betrayal of trust is one of...,Theres three basic reasons that this is frowne...
779,t3_54wr8w,You shouldnt have to pay more taxes just becau...,Im a firm believer in a meritocracy.Financiall...,Thats pretty much the point everyone wants fre...
835,t3_2lqcby,Catcalling and Street Harassment should be a t...,With the recent discussion of Street Harassmen...,Well theres a pretty big free speech problem. ...
1118,t3_4n5nmv,There is a difference between being happy with...,I came across the instagram user glitterandlaz...,Society doesnt do anything to help anorexics b...
1156,t3_1sw6ya,As a proud liberal I believe that framing left...,I believe that leftwingliberal parties are not...,They support more government involvement in pe...
1239,t3_2131f9,Banning picketing outside funerals is a good i...,Its hard to counterargue slippery slope critic...,Youre sort of right but I think for the wrong ...
1448,t3_2kk7pc,Revenge porn should not be a criminal offence.,Revenge porn can be extremely hurtful there is...,In several highprofile cases where gossip has ...
1590,t3_5cy2u9,Porn only brings harm and should be banned.,Im not saying from a specific country or regio...,Youre forgetting that porn is a job that helps...


In [137]:
# Output JSON List .jsonl
import json

data = []
for idx, row in args_clean_df.iterrows():
    data.append({
        "id": row["id"],
        "claim": row["claim"],
        "argument": row["argument"],
        "counter": row["counter"]
    })

with open("./data/cmv_cleaned.jsonl", "w", encoding='utf-8') as f:
    for d in data:
        f.write(json.dumps(d))
        f.write("\n")